# Deliverable 2. Create a Customer Travel Destinations Map

----

1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Geoapify API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found".

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame. In the point for each city add:

    * The city name
    
    * The country code
    
    * The weather description
    
    * The point's size should be the maximum temperature for the city

15. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.

---

## Ensure that the dependencies and the Geoapify API key is imported correctly

In [47]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Turn off warning messages
import warnings
warnings.filterwarnings("ignore")

# Import API key
from config import geoapify_key

# Configure gmaps
gmaps.configure(api_key=geoapify_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`

In [48]:
# Set the file path to import the WeatherPy_database.csv file.
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame.
city_data_df = pd.read_csv(file_path)

# Display sample data.
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Harbour Breton,CA,47.4833,-55.7983,31.87,79,100,13.11,overcast clouds
1,1,Ribeira Grande,PT,38.5167,-28.7000,60.78,73,98,20.00,overcast clouds
2,2,Anadyr,RU,64.7500,177.4833,-29.00,73,100,11.18,overcast clouds
3,3,Punta Arenas,CL,-53.1500,-70.9167,37.51,75,0,11.50,clear sky
4,4,Atuona,PF,-9.8000,-139.0333,78.66,76,12,13.38,few clouds
5,5,Clyde River,CA,70.4692,-68.5914,-23.57,76,100,21.85,light snow
6,6,Souillac,MU,-20.5167,57.5167,82.78,65,75,6.91,broken clouds
7,7,Rikitea,PF,-23.1203,-134.9692,79.41,72,67,5.77,broken clouds
8,8,Port Hardy,CA,50.6996,-127.4199,39.38,87,100,25.32,light rain
9,9,Husavik,IS,66.0449,-17.3389,27.70,77,100,6.15,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation

In [49]:
# Prompt the user to enter minimum and maximum temperature criteria.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria.
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`

In [50]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]

# Display sample data
preferred_cities_df

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Atuona,PF,-9.8000,-139.0333,78.66,76,12,13.38,few clouds
6,6,Souillac,MU,-20.5167,57.5167,82.78,65,75,6.91,broken clouds
7,7,Rikitea,PF,-23.1203,-134.9692,79.41,72,67,5.77,broken clouds
10,10,Faanui,PF,-16.4833,-151.7500,82.31,73,5,11.79,clear sky
12,12,Bambous Virieux,MU,-20.3428,57.7575,83.21,65,75,6.91,broken clouds
...,...,...,...,...,...,...,...,...,...,...
672,672,Mitsamiouli,KM,-11.3847,43.2844,82.94,70,100,7.61,overcast clouds
673,673,Catamarca,AR,-28.4696,-65.7852,82.78,55,0,1.99,clear sky
674,674,Longido,TZ,-2.7333,36.6833,77.49,49,20,10.09,few clouds
677,677,Sharjah,AE,25.3573,55.4033,80.89,21,0,6.91,clear sky


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows

In [51]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data.
clean_travel_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
4,4,Atuona,PF,-9.8000,-139.0333,78.66,76,12,13.38,few clouds
6,6,Souillac,MU,-20.5167,57.5167,82.78,65,75,6.91,broken clouds
7,7,Rikitea,PF,-23.1203,-134.9692,79.41,72,67,5.77,broken clouds
10,10,Faanui,PF,-16.4833,-151.7500,82.31,73,5,11.79,clear sky
12,12,Bambous Virieux,MU,-20.3428,57.7575,83.21,65,75,6.91,broken clouds
13,13,East London,ZA,-33.0153,27.9116,70.74,94,75,2.30,broken clouds
17,17,Okitipupa,NG,6.5047,4.7839,74.23,71,92,2.95,overcast clouds
20,20,Kelo,TD,9.3086,15.8066,70.11,19,95,9.26,overcast clouds
23,23,Irece,BR,-11.3042,-41.8558,71.73,64,82,7.36,broken clouds
27,27,Najran,SA,17.4924,44.1277,74.05,25,4,2.17,clear sky


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng"

In [52]:
# Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_travel_cities[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]]

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng
4,Atuona,PF,78.66,few clouds,-9.8000,-139.0333
6,Souillac,MU,82.78,broken clouds,-20.5167,57.5167
7,Rikitea,PF,79.41,broken clouds,-23.1203,-134.9692
10,Faanui,PF,82.31,clear sky,-16.4833,-151.7500
12,Bambous Virieux,MU,83.21,broken clouds,-20.3428,57.7575
13,East London,ZA,70.74,broken clouds,-33.0153,27.9116
17,Okitipupa,NG,74.23,overcast clouds,6.5047,4.7839
20,Kelo,TD,70.11,overcast clouds,9.3086,15.8066
23,Irece,BR,71.73,broken clouds,-11.3042,-41.8558
27,Najran,SA,74.05,clear sky,17.4924,44.1277


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame

In [53]:
# Create a new column "Hotel Name".
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Atuona,PF,78.66,few clouds,-9.8000,-139.0333,
6,Souillac,MU,82.78,broken clouds,-20.5167,57.5167,
7,Rikitea,PF,79.41,broken clouds,-23.1203,-134.9692,
10,Faanui,PF,82.31,clear sky,-16.4833,-151.7500,
12,Bambous Virieux,MU,83.21,broken clouds,-20.3428,57.7575,
13,East London,ZA,70.74,broken clouds,-33.0153,27.9116,
17,Okitipupa,NG,74.23,overcast clouds,6.5047,4.7839,
20,Kelo,TD,70.11,overcast clouds,9.3086,15.8066,
23,Irece,BR,71.73,broken clouds,-11.3042,-41.8558,
27,Najran,SA,74.05,clear sky,17.4924,44.1277,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city

In [54]:
# Set parameters to search for a hotel
radius = 5000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

## Use a for loop to iterate through the `hotel_df` DataFrame. Retrieve the latitude and longitude of each city and use the Geoapify API to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, set the hotel name as "No hotel found"

In [71]:
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from DataFrame.
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set up the base URL for the Geoapify Places API.
    base_url = "https://api.geoapify.com/v2/places"

    # Make request and retrieve the JSON data by using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found set the hotel name as "No hotel found".
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
Atuona - nearest hotel: No hotel found
Souillac - nearest hotel: No hotel found
Rikitea - nearest hotel: No hotel found
Faanui - nearest hotel: No hotel found
Bambous Virieux - nearest hotel: No hotel found
East London - nearest hotel: No hotel found
Okitipupa - nearest hotel: No hotel found
Kelo - nearest hotel: No hotel found
Irece - nearest hotel: No hotel found
Najran - nearest hotel: No hotel found
Hithadhoo - nearest hotel: No hotel found
Noumea - nearest hotel: No hotel found
Kieta - nearest hotel: No hotel found
Sompeta - nearest hotel: No hotel found
Arraial Do Cabo - nearest hotel: No hotel found
Busselton - nearest hotel: No hotel found
Namibe - nearest hotel: No hotel found
Sirari - nearest hotel: No hotel found
Vaini - nearest hotel: No hotel found
Visakhapatnam - nearest hotel: No hotel found
Sinnamary - nearest hotel: No hotel found
Butaritari - nearest hotel: No hotel found
Mareeba - nearest hotel: No hotel found
San Patricio - nearest hotel: No ho

Poso - nearest hotel: No hotel found
Santiago De Cao - nearest hotel: No hotel found
Qui Nhon - nearest hotel: No hotel found
Mahajanga - nearest hotel: No hotel found
Luganville - nearest hotel: No hotel found
Majene - nearest hotel: No hotel found
Jesus Carranza - nearest hotel: No hotel found
Dwarka - nearest hotel: No hotel found
Lobito - nearest hotel: No hotel found
Sumenep - nearest hotel: No hotel found
Cururupu - nearest hotel: No hotel found
Quatre Cocos - nearest hotel: No hotel found
Ambilobe - nearest hotel: No hotel found
Ayolas - nearest hotel: No hotel found
Bima - nearest hotel: No hotel found
Vao - nearest hotel: No hotel found
Gwadar - nearest hotel: No hotel found
Aripuana - nearest hotel: No hotel found
Mount Gambier - nearest hotel: No hotel found
Mana - nearest hotel: No hotel found
Tautira - nearest hotel: No hotel found
Santa Isabel - nearest hotel: No hotel found
Carauari - nearest hotel: No hotel found
Marsh Harbour - nearest hotel: No hotel found
Gamba - nea

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
4,Atuona,PF,78.66,few clouds,-9.8000,-139.0333,No hotel found
6,Souillac,MU,82.78,broken clouds,-20.5167,57.5167,No hotel found
7,Rikitea,PF,79.41,broken clouds,-23.1203,-134.9692,No hotel found
10,Faanui,PF,82.31,clear sky,-16.4833,-151.7500,No hotel found
12,Bambous Virieux,MU,83.21,broken clouds,-20.3428,57.7575,No hotel found
...,...,...,...,...,...,...,...
672,Mitsamiouli,KM,82.94,overcast clouds,-11.3847,43.2844,No hotel found
673,Catamarca,AR,82.78,clear sky,-28.4696,-65.7852,No hotel found
674,Longido,TZ,77.49,few clouds,-2.7333,36.6833,No hotel found
677,Sharjah,AE,80.89,clear sky,25.3573,55.4033,No hotel found


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`

In [74]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")

# Display sample data
clean_hotel_df.head(10)

,City_ID,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,2,Aksu,CN,82.42,clear sky,41.12,80.26,Pudong Holiday Hotel
1,7,Touros,BR,80.60,broken clouds,-5.20,-35.46,INN NEW HORIZON
2,10,Morehead,US,75.20,clear sky,37.27,-87.18,CCI Express Inn
3,11,Port Elizabeth,ZA,84.20,clear sky,-33.92,25.57,39 On Nile Guest House
4,16,Northam,GB,82.40,few clouds,51.03,-4.22,Durrant House Hotel
5,18,Hyderabad,IN,87.01,haze,17.38,78.47,"Taj Krishna, Hyderabad"
6,20,Port Alfred,ZA,82.00,clear sky,-33.59,26.89,The Halyards Hotel
7,22,Atuona,PF,79.72,clear sky,-9.80,-139.03,Villa Enata
8,23,Kapaa,US,73.40,heavy intensity rain,22.08,-159.32,Sheraton Kauai Resort at Coconut Beach
9,25,Nikolskoye,RU,88.00,clear sky,59.70,30.79,Tourist House - Sablino


## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the `Vacation_Search`` folder

In [75]:
# Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

## Use GeoViews to create a map that displays a point for every city in the `clean_hotel_df` DataFrame.

In the point for each city add:

* The city name
* The country code
* The weather description
* The point's size should be the maximum temperature for the city

In [43]:
# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE